# Example Chapter 5: Accessing Data from the Internet
In this example we tap into the yfinance library to get monthly returns on the Russell 3000 index (^RUT) and Ford Motor Company (F).  We also pull data from and Ken French's website on short-term US t-bill returns. However, it would be difficult to determine that we're using these data sources in the code below. To make the process of gathering data from these sources as easy as possible, we package most of the code in easy-to-use functions located in the file `bates\_boyer\_fletcher\_functions.py`.  We then use these data to estimate the annualized expected return, volatility, and Sharpe ratio for the Russell 3000 and Ford.  Try changing the tickers and date ranges to pull data for different companies over different time periods.

### Imports and Setup

In [ ]:
# Load in bates_boyer_fletcher_functions.py from the GitHub repository 
!curl -O https://raw.githubusercontent.com/boyerb/Investments/refs/heads/master/functions/bates_boyer_fletcher_functions.py

In [ ]:
# import the functions from the file we just downloaded
import bates_boyer_fletcher_functions as invf 
# import other necessary libraries
import numpy as np


### Access Monthly Returns 
In this block we access monthly returns for a list of two tickers, `ticker = ['^RUA', 'F']`, where `^RUA` is the ticker for the Russell 3000 index and `F` is the ticker for Ford Motor Company. The key function is `get_monthly_returns`. 

`get_monthly returns` has the following inputs:
- **ticker** = list of desired tickers tht looks like this: `['ticker1', 'ticker2', ..., 'tickerN']` 
- **begin** = beginning month in format yyyy-mm-dd
- **end** = ending month in format yyyy-mm-dd
- **tbill_return** = True or False. If True, the return on short-term tbills will be included in the DataFrame. If False, the return on short-term tbills will not be included.
The output is a DataFrame with monthly returns for each ticker, and the return on short-term tbills if `tbill_return=True`.

- We then calculate the annualized average return and volatility.

In [ ]:
# Get monthly returns for a given list of stock tickers using the yfinance API
ticker = ['^RUA', 'F']
# Use tbill_return=True to get the return on short-term tbills, or False to exclude tbill returns
dat = invf.get_monthly_returns(ticker, '1990-01-01', '2023-12-31', tbill_return=False)
print(dat.head())

In [ ]:
# Calculate the annualized average return and volatility
A_Avg = dat.mean() * 12
A_Vol = dat.std() * np.sqrt(12)
print('Anualized Average Return')
print(A_Avg)
print()
print('Annualized Volatility')
print(A_Vol)

### Calculate Sharpe Ratios
In this block we access data for the Russell 3000 index and Ford Motor Company **ticker = ['^RUA', 'F']**. We also retrieve data on short-term t-bill returns. Using these data we calculate the Sharpe ratios for the Russell 3000 and Ford.

In [ ]:
# Download data along with return on short-term tbills from Ken French's website
dat = invf.get_monthly_returns(ticker, '1990-01-01', '2023-12-31', tbill_return=True)
# Calculate excess returns
dat['exret0'] = dat[ticker[0]] - dat['RF']
dat['exret1'] = dat[ticker[1]] - dat['RF']
print(dat.head())

In [ ]:
# Calculate and print Sharpe ratios
A_AVG_exret0 = dat['exret0'].mean() * 12
A_Vol_exret0 = dat[ticker[0]].std() * np.sqrt(12)
Sharpe0 = A_AVG_exret0/A_Vol_exret0

A_AVG_exret1 = dat['exret1'].mean() * 12
A_Vol_exret1 = dat[ticker[1]].std() * np.sqrt(12)
Sharpe1 = A_AVG_exret1/A_Vol_exret1

print('Sharpe Ratios')
print(ticker[0], Sharpe0)
print(ticker[1], Sharpe1)